In [434]:
import pandas as pd

In [435]:
import matplotlib.pyplot as plt


QUE ES CADA COLUMNA, formato, y si esta presente en otro DF (con mismo o distinto nombre)

AUCTIONS
auction_type_id: todo NaN, DESCARTADA
country: todos mismo pais, DESCARTADA
date: fecha
device_id: ID del dispositivo. es equivalente a ref_hash de clicks e installs
platform: plataforma. tiene 2 valores posibles asi que debe ser ios y android
ref_type_id: ¿?  tiene MISMOS 2 valores posibles que platform
source_id:  5 valores posibles, ni idea que es
    

    
CLICKS
advertiser_id: 7 valores posibles. hay un unico que es la mayoria de las veces
action_id: todo NaN, DESCARTADA
source_id: 11 valores posibles. ni idea
created: fecha creacion
country_code: todos mismo pais, DESCARTADA
latitude: latitud
longitude: longitud 
wifi_connection: todos false. DESCARTADA
carrier_id: ni idea que es carrier. tiene varios valores posibles
trans_id: supongo que id de transaccion? son todos unicos
os_minor: ni idea,  varios valores posibles
agent_device: ???
os_major: 10 valores posibles, ni idea
specs_brand: 5 valores posibles. 2 con muchisima cantidad y 3 con re pocas veces. marca de algo?
brand: marca de algo?
timeToClick: ¿tiempo que tarda en clickear desde que aparece? en segundos
touchX:  ¿posicion de la pantalla en el eje horizontal?
touchY:  ¿posicion de la pantalla en el eje vertical?
ref_type: 4 valores posibles. ni idea
ref_hash: id del dispositivo. equivalente a device id de auctions y ref hash de installs
    
    
EVENTS
date: date
event_id: id del evento
ref_type:  apple_ifa o google_advertising_id. supongo que el que tiene mas cantidad es google
ref_hash: no se que es pero muchos valores se repiten. algunos mas de 2000 veces. varios individuales tamb
application_id: id de app supongo? la explicacion en el drive es rara
attributed: cuantas subastas gano jampp. booleano
device_countrycode: todos repetidos. descartar
device_os_version: version del sistema operativo? hay muchos y muchos repetidos
device_brand: marca del dispositivo? medio raro porque hay muchas marcas que tienen solo 1 registro
device_model: modelo del celular, varios repetidos
device_city: ciudad, varios repetidos
session_user_agent: hash de la sesion?
trans_id: inutil. Todos NaN
user_agent: 
event_uuid:
carrier: suponemos que es empresa de telefonia pero no tieen mucho sentid oa cantidad de valores posibles
kind:
device_os: sistema operativo del dispositivo
wifi: si estan con wifi o no
connection_type: si esta por cable o por antena de 4g
ip_address: ip? se puede agrupar. raro que algunos se repitan mucho
device_language: 
    

    
INSTALLS
created: -
application_id:
ref_type:
ref_hash:
click_hash:
attributed: todo nan ,inutil
implicit:
device_countrycode:
device_brand: marca del dispositivo
device_model: modelo del dispositivo
session_user_agent: user agent. La mayoria son http-kit. En general no es util
user_agent:
event_uuid:
kind:
wifi: true or false si esta con wifi o no
trans_id: inutil. Todos NaN
ip_address: 
device_language: hay 184 idiomas ?

In [436]:
auctions = pd.read_csv('auctions.csv')

MemoryError: 

In [ ]:
clicks = pd.read_csv('clicks.csv')

In [ ]:
events = pd.read_csv('events.csv')

In [ ]:
installs = pd.read_csv('installs.csv')

In [ ]:
auctions.head()

In [ ]:
clicks.head()

In [ ]:
events.head()

In [ ]:
events["connection_type"].value_counts()

In [ ]:
installs.head()

In [ ]:
# Un solo pais
auctions["country"].unique()

In [ ]:
# INUTIL
clicks['wifi_connection'].unique()

In [ ]:
# UTIL
installs['wifi'].value_counts()

In [ ]:
installs.columns

In [ ]:
list(installs.columns)

In [ ]:
installs_columns = list(installs.columns)
installs_columns.remove("created")
for i in installs_columns:
    print(i," ",installs[i].value_counts())
    
# Importante en 

In [ ]:
installs.info()

In [ ]:
#solo 7 de los clicks coinciden en ref hash con installs. el resto de los installs no fueron nuestros

In [ ]:
installs["ref_hash"].isin(clicks["ref_hash"]).value_counts()

In [ ]:
#miro las plataformas . varios tienen informacion del modelo de celular, version de android, etc

In [ ]:
installs["user_agent"].value_counts().to_frame().iloc[50:100]

In [ ]:
#android = android
#darwin = MAC OS

#creo columnas sobre sistema operativo 

In [ ]:
import numpy as np

In [ ]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [ ]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [ ]:
installs["os"].value_counts()

In [ ]:
#la mayoria de las instalaciones son con wifi

In [ ]:
installs["wifi"].value_counts()

In [ ]:
#agrego la marca del celular en caso de installs con android

In [ ]:
android = installs[installs["os"] == "android"]

In [ ]:
android.head()["user_agent"]

In [ ]:
def brand_of_android_device(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "sm-" in agent or "gt-" in agent:
        return "samsung"
    elif "lg-" in agent or "lm-" in agent:
        return "lg"
    elif "moto" in agent or "xt" in agent:
        return "motorola"
    elif "miui" in agent or " mi " in agent:
        return "xiaomi"
    elif "asus" in agent:
        return "asus"
    elif "lenovo" in agent:
        return "lenovo"
    elif  "huawei" in agent:
        return "huawei"
    elif "oneplus" in agent:
        return "oneplus"
    elif "positivo" in agent or "S455" in agent:
        return "positivo bgh"
    elif "zte" in agent:
        return "zte"
    elif "nokia" in agent:
        return "nokia"
    else:
        return "other"

In [ ]:
android["brand"] = android.apply(brand_of_android_device, axis=1)

In [ ]:
android["brand"].value_counts()

In [ ]:
#que carajo son los mac os? 

In [ ]:
installs[installs["os"] == "mac os"]["user_agent"].apply(lambda x: x.split("/")[0]).value_counts()

In [ ]:
#todos son http-kit/2.0

In [ ]:
installs[installs["os"] == "mac os"][["user_agent", "session_user_agent"]].head()

In [ ]:
#cosas latitud y longitud
#los datos no dicen un carajo
#no llega a nada, al ejecutar el geolocator 

In [ ]:
lat = clicks.iloc[1]["latitude"]
lon = clicks.iloc[1]["longitude"]

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
"""geolocator = Nominatim()
geolocator.reverse('52.5094982,13.3765983')"""

In [ ]:
"""latlon = str(lon)+", "+str(lat)
geolocator.reverse(latlon)"""

In [ ]:
# Clicks by latitude and longitude
clicks[["latitude", "longitude"]].groupby(["latitude", "longitude"]).size().reset_index().head()


In [ ]:
# Hay solo 82 latitudes y longitudes que hacen clicks.
len(clicks[["latitude", "longitude"]].groupby(["latitude", "longitude"]).size().reset_index())

In [ ]:
# 26351
len(clicks)

In [ ]:
# Clicks

In [ ]:
clicks['range'] = pd.to_datetime(clicks['created']).dt.floor('60T').dt.time

In [ ]:
clicksIn60Min = clicks.groupby(["range"])["range"].count()

In [ ]:
clicksIn60Min

In [ ]:
clicksIn60Min.plot.bar()

In [ ]:
#muchos eventos corresponden a un mismo click

In [ ]:
events["ref_hash"].isin(clicks["ref_hash"]).value_counts()

In [ ]:
clicks["ref_hash"].isin(events["ref_hash"]).value_counts()

In [ ]:
#me fijo si para algun ref_hash de evento hay mas de un click

In [ ]:
#en este caso es asi
clicks[clicks["ref_hash"]==71063745129569882]

In [ ]:
#para esos 2 clicks, hay 13 eventos. pueden ser dos sesiones distintas
events[events["ref_hash"] == 71063745129569882].shape

In [ ]:
#junto por ref hash, me quedo solo con los hashes que aparecen en ambos dataset

In [ ]:
events_clicks = events.merge(clicks,on="ref_hash", how="inner")

In [ ]:
#agrupo por ref hash , cuento y agarro alguna columna cualquiera para ver cuantos eventos tengo por un ref hash 
#que tambien aparece al menos una vez en clicks

In [ ]:
counted_events_by_click = events_clicks.groupby("ref_hash").count()["attributed"]

In [ ]:
counted_events_by_click.sort_values(ascending=False).head()

In [ ]:
#algunos hashes tienen muchos eventos. miro si los primeros tieenen muchos clicks

In [ ]:
#el ref hash que aparece en ambos con mas eventos tiene solo 3 clicks
len(clicks[clicks["ref_hash"] == 5639173013143339138])

In [ ]:
#miro cuantos installs tiene
#nada
len(installs[installs["ref_hash"] == 5639173013143339138])

In [ ]:
#el segundo ref hash que aparece en ambos con mas eventos tambien tiene 3
len(clicks[clicks["ref_hash"] == 8569678576097759996])

In [ ]:
#tampoco tiene installs
len(installs[installs["ref_hash"] == 8569678576097759996])

In [ ]:
#conclusion? son bots que nunca llegan a instalar?

In [ ]:
#comparamos si en algun moemnto estan cerca la fecha del click con la de alguno de los eventos para ese hash
#es posible que ningun evento corresponda a un click? que no sea considerado evento?

In [ ]:
#miramos por ejemplo un hash que tiene 1 click y 4 eventos
#no coinciden. los eventos tienen todos al menos 2 dias de diferencia con el click
events_clicks[events_clicks["ref_hash"] == 6251876360487092][["created","date"]]

In [ ]:
#hay muchos casos con 1 solo evento y 1 solo click. miro si en algunos de  ellos coinciden las fechas
counted_events_by_click.head(10)

In [ ]:
#no coincide
events_clicks[events_clicks["ref_hash"] == 4467013148049275][["created","date"]]

In [ ]:
#tiene 3 hs de diferencia. si click estuviera en UTC y date en gmt -3 en ese caso tendrian solo 9 minuts de diferencia
events_clicks[events_clicks["ref_hash"] == 74885721366447582][["created","date"]]

In [ ]:
#no coincide
events_clicks[events_clicks["ref_hash"] == 16228483706113027][["created","date"]]

In [ ]:
comparing = events_clicks[["ref_hash","date","created"]]

In [ ]:
comparing.head()

In [ ]:
#mirando el caso de 8187571300120427764
#para un mismo click hay distintos eventos
#lo que no encuentro sentido es que los eventos son todos anteriores al click.
#incluso asumiendo que created esta en utc y lo convertimos a gmt -3, casi todos los eventos igual serian
#anteriores excepto por el dei ndice 0. el cua seria 5 minutos despues
#hago columna con la diferencia en tiempo entre evento y click
events_clicks['date_dt'] = pd.to_datetime(events_clicks.date, utc=True)
events_clicks['created_dt'] = pd.to_datetime(events_clicks.created, utc=True)
events_clicks['event_click_delta'] = (events_clicks.date_dt - events_clicks.created_dt)

In [ ]:
#miro algunos ejemplos

In [ ]:

events_clicks[events_clicks.ref_hash ==6251876360487092][['event_id', 'date', 'created', 'event_click_delta']].sort_values('event_click_delta')

In [ ]:
events_clicks[events_clicks.ref_hash == 8187571300120427764][['event_id', 'date', 'created', 'event_click_delta']].sort_values('event_click_delta')

In [ ]:
events_clicks[events_clicks.ref_hash == 56820982416916093][['event_id', 'date', 'created', 'event_click_delta']].sort_values('event_click_delta')

In [ ]:
events_clicks[events_clicks.ref_hash == 106694467223485662][['event_id', 'date', 'created', 'event_click_delta']].sort_values('event_click_delta')

In [ ]:
#miro si tengo para ese ref hash infomacion en auctions (el feature equivalente es device_id)

In [ ]:
auctions[auctions["device_id"] == 8187571300120427764]

In [ ]:
#ahora busco si hay alguna relacion entre evento e install
#es una instalacion un evento?
#hay un patron de eventos previos a la instalacion?


In [ ]:
#puede haber mas de un install para un ref hash?
#vemos que la gran mayoria tienen solo 1 install
installs.groupby(["ref_hash"]).count()["created"].value_counts()

In [ ]:
#junto installs y events y convierto
#tambien creo nueva columna para ver la diferencia de tiempo entre eventos e instalacion
events_installs = events.merge(installs, on="ref_hash", how="inner")
events_installs["date"] = pd.to_datetime(events_installs["date"])
events_installs["created"] = pd.to_datetime(events_installs["created"])
events_installs['event_install_delta'] = (events_installs.date - events_installs.created)

In [ ]:
#pudimos mergear 53157 filas

In [ ]:
events_installs.shape

In [ ]:
#pero hay que ver por cada install cuantos eventos hubo

In [ ]:
#algunos casos tienen muchismos eventos por instalacion
events_installs["ref_hash"].value_counts().head()

In [ ]:
#busquemos casos con menos eventos para poder analizar mas comodamente
events_installs["ref_hash"].value_counts()[240:255]

In [ ]:
events_installs["ref_hash"].value_counts()[400:420]

In [ ]:
installs[installs["ref_hash"] == 1688171320462340261]

In [ ]:
#date es de events y created es de installs 

In [ ]:
events_installs_dates = events_installs[["ref_hash", "date", "created", "event_install_delta", "event_id"]]
events_installs_dates.head(10)

In [ ]:
#analizamos un caso con 19 eventos. hubo solo 1 install
#en la filas 5,6,y 7 los dates parecen coincidir en el tiempo donde fue isntalado (created)
#ninguno es exacto, pero los 3 tienen de event id al 22.
#¿el install es el evento 22?
#los ordeno por fecha del evento para comparar

#todas las secuencias de eventos parecen arrancar con el ID 1, y continuan con el 33
#sera 1 el evento que se entra a la publicidad?

In [ ]:
events_installs_dates[events_installs_dates["ref_hash"] == 5007008548832677915].sort_values(by="date")

In [ ]:
#analizamos con otro ref hash
#9137998220583496972 tiene 71 eventos
#nuevamente el evento 33 es el mas comun 
#miramos los ultimos 10 porque son los mas cercanos a la instalacion
#mirando las diferencias de tiempo , parece ser que el evento 1 sigue siendo el principio de cada secuencia de eventos
#cada vez que hay un evento 1, el evento anterior en tiempo es lejano, asi que debe ser otra "sesion"
#excepto por el caso de la fila 24, que arranca con 80. hay que ver otros casos similares

In [ ]:
events_installs_dates[events_installs_dates["ref_hash"] == 9137998220583496972].sort_values(by="date").tail(10)

In [ ]:
#en este caso todos los eventos son 22, pero parecen corresponder a la misma sesion

In [ ]:
events_installs_dates[events_installs_dates["ref_hash"] == 3825524615010901958].sort_values(by="date")

In [ ]:
#en este caso no se arranca la sesion con evento 1 sino con el 80.
#el evento 22 vuelve a parecer ser el de la instalacion
events_installs_dates[events_installs_dates["ref_hash"] == 2808800821029079796].sort_values(by="date")

In [ ]:
#cuales son los eventos mas comunes?

events_installs["event_id"].value_counts().head(10)

In [ ]:
#Miremos los event id mas comunes ,en un rango de 1 minuto desde la instalacion hacia atras

In [ ]:
from datetime import timedelta
ten_minutes_delta = timedelta(minutes=10)
half_hour_delta = timedelta(minutes=30)
two_hour_delta = timedelta(minutes=120)
five_hour_delta = timedelta(minutes=300)
one_day_delta = timedelta(days=1)

In [ ]:
events_installs_dates[events_installs_dates["event_install_delta"] < one_hour_delta].head()

In [ ]:
#miro en distintas distancias de tiempo desde la instalacion

In [ ]:
#los 10 minuots mas cercanos a la instalacion tienen gran mayoria de eventos 22
#aparece mucho el evento 138 

In [ ]:
events_installs_dates[events_installs_dates["event_install_delta"] < ten_minutes_delta]["event_id"].value_counts().head()

In [ ]:
#si agrando a media hora mas cercana, la cantidad de eventos nro 138 es la misma casi
#pero crece mucho el evento 22

In [ ]:
events_installs_dates[events_installs_dates["event_install_delta"] < half_hour_delta]["event_id"].value_counts().head()

In [ ]:
#miro los eventos mas comunes con mas de 2 horas de diferencia con la instalacion
#el porcentaje de eventos 22 es menor, pero sigue siendo la mayoria
#desaparece el evento 138

In [ ]:
events_installs_dates[events_installs_dates["event_install_delta"] > two_hour_delta]["event_id"].value_counts().head()

In [ ]:
#con 5 horas o mas de diferencia el evento 22 sigue sienod el principal pero su porcentaje respecto al resto sigue bajando

In [ ]:
events_installs_dates[events_installs_dates["event_install_delta"] > five_hour_delta]["event_id"].value_counts().head()

In [ ]:
#con 1 dia o mas es lo mismo que con 5 horas o mas

In [ ]:
events_installs_dates[events_installs_dates["event_install_delta"] > one_day_delta]["event_id"].value_counts().head()

In [ ]:
#al ser el 138 un evento que solo aparece bien cerca del momento de instalacion, y el 22 
#un evento que aparece en proporcion muchisimo mas cuando mas nos acercamos a la instalacion
#tal vez el evento 22 sea un paso previo muy cercano a la instalacion
#y el evento 138 sea la instalacion en si. no creo que sea casualidad que la cantidad total
#de eventos 138 tenga una cifra muy cercana al tamaño en filas del csv de installs
#hay que seguir analizando